In [1]:
import pandas as pd
import os

In [72]:

base_dir = '/Users/jonaheaton/Desktop/alignment_analysis/Output_Data'
data_dir = os.path.join(base_dir,'ST001849_hilic_pos_vs_ST001428_hilic_pos')
comparison_id = 'ST001849_hilic_pos_ST001428_hilic_pos'

name0 = 'ST001849_hilic_pos'
name1 = 'ST001428_hilic_pos'
method1 = 'Eclipse_50_50'
method2 = 'metabCombiner_50_50_Jan23'

align_res1_path = os.path.join(data_dir,comparison_id + '_' + method1 + '_alignment_result.csv')
align_res2_path = os.path.join(data_dir,comparison_id + '_' + method2 + '_alignment_result.csv')

align_res1 = pd.read_csv(align_res1_path, index_col=0)
align_res2 = pd.read_csv(align_res2_path, index_col=0)

In [73]:
align_res1

,ST001849_hilic_pos,ST001428_hilic_pos
0,FT2076,FT2109
1,FT2838,FT3141
2,FT943,FT792
3,FT3210,FT3687
4,FT6071,FT7882
...,...,...
872,FT807,FT667
873,FT6891,FT8849
874,FT6436,FT8314
875,FT5047,FT6438


In [74]:
align_res2

,ST001849_hilic_pos,ST001428_hilic_pos,score
0,FT198,FT25,0.6361
1,FT237,FT66,0.7139
2,FT207,FT51,0.6813
3,FT211,FT43,0.6266
4,FT277,FT74,0.7465
...,...,...,...
414,FT6959,FT8936,0.5721
415,FT6962,FT8940,0.6474
416,FT6965,FT8952,0.6006
417,FT6968,FT8948,0.5430


In [75]:
align_res1[name0] = align_res1[name0].apply(lambda x: name0 + '_' + str(x))
align_res1[name1] = align_res1[name1].apply(lambda x: name1 + '_' + str(x))
align_res2[name0] = align_res2[name0].apply(lambda x: name0 + '_' + str(x))
align_res2[name1] = align_res2[name1].apply(lambda x: name1 + '_' + str(x))


In [79]:
import networkx as nx

# Create a new bipartite graph
G = nx.Graph()

# Add nodes from the first column of align_res1 as one set of nodes
G.add_nodes_from(align_res1[name0], bipartite=0)
G.add_nodes_from(align_res2[name0], bipartite=0)

# Add nodes from the second column of align_res1 as another set of nodes
G.add_nodes_from(align_res1[name1], bipartite=1)
G.add_nodes_from(align_res2[name1], bipartite=1)

# Add edges between the nodes
G.add_edges_from(zip(align_res1[name0], align_res1[name1]), weight=1)
G.add_edges_from(zip(align_res2[name0], align_res2[name1]), weight=2)


# Create a new graph to store the result
H = nx.Graph()

num_components = nx.number_connected_components(G)
# Find maximum cardinality matching for each connected component
for component in nx.connected_components(G):
    subgraph = G.subgraph(component)
    num_size = len(subgraph)
    matching = nx.max_weight_matching(subgraph)
    # if num_size > 2:
        # print(num_size)
        # break
    H.add_edges_from(tuple(edge) for edge in matching)


# create a new dataframe to store the result
result = pd.DataFrame(columns=[name0, name1])

# add the edges from the matching to the dataframe
result[name0] = [edge[0] if edge[0].startswith(name0) else edge[1] for edge in H.edges()]
result[name1] = [edge[0] if edge[0].startswith(name1) else edge[1] for edge in H.edges()]


In [82]:
# Check if the result matches align_res1
result_align_res1 = align_res1[align_res1[name0].isin(result[name0]) & align_res1[name1].isin(result[name1])]
print("Result matches align_res1:", result_align_res1.equals(align_res1))

# Check if the result matches align_res2
result_align_res2 = align_res2[align_res2[name0].isin(result[name0]) & align_res2[name1].isin(result[name1])]
print("Result matches align_res2:", result_align_res2.equals(align_res2))

# the highest ranking method should have the most

Number of results from align_res1: 831
Result matches align_res1: False
Number of results from align_res2: 419
Result matches align_res2: True


In [90]:
align_res2

,ST001849_hilic_pos,ST001428_hilic_pos,score
0,ST001849_hilic_pos_FT198,ST001428_hilic_pos_FT25,0.6361
1,ST001849_hilic_pos_FT237,ST001428_hilic_pos_FT66,0.7139
2,ST001849_hilic_pos_FT207,ST001428_hilic_pos_FT51,0.6813
3,ST001849_hilic_pos_FT211,ST001428_hilic_pos_FT43,0.6266
4,ST001849_hilic_pos_FT277,ST001428_hilic_pos_FT74,0.7465
...,...,...,...
414,ST001849_hilic_pos_FT6959,ST001428_hilic_pos_FT8936,0.5721
415,ST001849_hilic_pos_FT6962,ST001428_hilic_pos_FT8940,0.6474
416,ST001849_hilic_pos_FT6965,ST001428_hilic_pos_FT8952,0.6006
417,ST001849_hilic_pos_FT6968,ST001428_hilic_pos_FT8948,0.5430


In [89]:
result_align_res2

,ST001849_hilic_pos,ST001428_hilic_pos,score
0,ST001849_hilic_pos_FT198,ST001428_hilic_pos_FT25,0.6361
1,ST001849_hilic_pos_FT237,ST001428_hilic_pos_FT66,0.7139
2,ST001849_hilic_pos_FT207,ST001428_hilic_pos_FT51,0.6813
3,ST001849_hilic_pos_FT211,ST001428_hilic_pos_FT43,0.6266
4,ST001849_hilic_pos_FT277,ST001428_hilic_pos_FT74,0.7465
...,...,...,...
414,ST001849_hilic_pos_FT6959,ST001428_hilic_pos_FT8936,0.5721
415,ST001849_hilic_pos_FT6962,ST001428_hilic_pos_FT8940,0.6474
416,ST001849_hilic_pos_FT6965,ST001428_hilic_pos_FT8952,0.6006
417,ST001849_hilic_pos_FT6968,ST001428_hilic_pos_FT8948,0.5430


In [88]:
align_res1

,ST001849_hilic_pos,ST001428_hilic_pos
0,ST001849_hilic_pos_FT2076,ST001428_hilic_pos_FT2109
1,ST001849_hilic_pos_FT2838,ST001428_hilic_pos_FT3141
2,ST001849_hilic_pos_FT943,ST001428_hilic_pos_FT792
3,ST001849_hilic_pos_FT3210,ST001428_hilic_pos_FT3687
4,ST001849_hilic_pos_FT6071,ST001428_hilic_pos_FT7882
...,...,...
872,ST001849_hilic_pos_FT807,ST001428_hilic_pos_FT667
873,ST001849_hilic_pos_FT6891,ST001428_hilic_pos_FT8849
874,ST001849_hilic_pos_FT6436,ST001428_hilic_pos_FT8314
875,ST001849_hilic_pos_FT5047,ST001428_hilic_pos_FT6438


In [87]:
result_align_res1

,ST001849_hilic_pos,ST001428_hilic_pos
0,ST001849_hilic_pos_FT2076,ST001428_hilic_pos_FT2109
1,ST001849_hilic_pos_FT2838,ST001428_hilic_pos_FT3141
2,ST001849_hilic_pos_FT943,ST001428_hilic_pos_FT792
3,ST001849_hilic_pos_FT3210,ST001428_hilic_pos_FT3687
4,ST001849_hilic_pos_FT6071,ST001428_hilic_pos_FT7882
...,...,...
872,ST001849_hilic_pos_FT807,ST001428_hilic_pos_FT667
873,ST001849_hilic_pos_FT6891,ST001428_hilic_pos_FT8849
874,ST001849_hilic_pos_FT6436,ST001428_hilic_pos_FT8314
875,ST001849_hilic_pos_FT5047,ST001428_hilic_pos_FT6438


In [85]:
align_res1[name1].isin(result[name1])

0      True
1      True
2      True
3      True
4      True
       ... 
872    True
873    True
874    True
875    True
876    True
Name: ST001428_hilic_pos, Length: 877, dtype: bool

In [83]:
result_align_res1

,ST001849_hilic_pos,ST001428_hilic_pos
0,ST001849_hilic_pos_FT2076,ST001428_hilic_pos_FT2109
1,ST001849_hilic_pos_FT2838,ST001428_hilic_pos_FT3141
2,ST001849_hilic_pos_FT943,ST001428_hilic_pos_FT792
3,ST001849_hilic_pos_FT3210,ST001428_hilic_pos_FT3687
4,ST001849_hilic_pos_FT6071,ST001428_hilic_pos_FT7882
...,...,...
872,ST001849_hilic_pos_FT807,ST001428_hilic_pos_FT667
873,ST001849_hilic_pos_FT6891,ST001428_hilic_pos_FT8849
874,ST001849_hilic_pos_FT6436,ST001428_hilic_pos_FT8314
875,ST001849_hilic_pos_FT5047,ST001428_hilic_pos_FT6438


In [ ]:
# count the number of final-edges that came from align_res1 and align_res2
num_edges_from_align_res1 = len(result_align_res1)
num_edges_from_align_res2 = len(result_align_res2)

# count the number of final-edges that came from align_res1 and align_res2


In [64]:
subgraph.edges()

EdgeView([('FT2895', 'FT3210'), ('FT3210', 'FT3687')])

In [65]:
matching

{('FT3210', 'FT2895')}

In [66]:
align_res1[align_res1[name0] == 'FT3210']

,ST001849_hilic_pos,ST001428_hilic_pos
3,FT3210,FT3687


In [68]:
align_res2[align_res2[name0] == 'FT2895']

,ST001849_hilic_pos,ST001428_hilic_pos,score
171,FT2895,FT3210,0.5667


In [53]:
align_res1[align_res1[name0] == 'ST001849_hilic_pos_FT5067']

,ST001849_hilic_pos,ST001428_hilic_pos
11,ST001849_hilic_pos_FT5067,ST001428_hilic_pos_FT6514


In [56]:
align_res2[align_res2[name0] == 'ST001849_hilic_pos_FT5067']

,ST001849_hilic_pos,ST001428_hilic_pos,score
275,ST001849_hilic_pos_FT5067,ST001428_hilic_pos_FT6525,0.7625


In [52]:
name0

'ST001849_hilic_pos'

In [48]:
align_res1.shape


(877, 2)

In [49]:
align_res2.shape

(419, 3)

In [70]:
len(H.edges()) 

859

In [42]:
# Create a new graph to store the result
H = nx.Graph()

num_components = nx.number_connected_components(G)
# Find maximum cardinality matching for each connected component
for component in nx.connected_components(G):
    subgraph = G.subgraph(component)
    num_size = len(subgraph)
    matching = nx.max_weight_matching(subgraph)
    if num_size > 2:
        print(num_size)
        break
    H.add_edges_from(matching.items())

AttributeError: 'set' object has no attribute 'items'

In [43]:
matching

{('ST001849_hilic_pos_FT2076', 'ST001428_hilic_pos_FT2109')}

In [21]:
matching = nx.bipartite.maximum_matching(G)
H = nx.Graph(matching)

AmbiguousSolution: Disconnected graph: Ambiguous solution for bipartite sets.

In [20]:
G.edges()

EdgeView([('ST001849_hilic_pos_FT2076', 'ST001428_hilic_pos_FT2109'), ('ST001849_hilic_pos_FT2838', 'ST001428_hilic_pos_FT3141'), ('ST001849_hilic_pos_FT943', 'ST001428_hilic_pos_FT792'), ('ST001849_hilic_pos_FT3210', 'ST001428_hilic_pos_FT3687'), ('ST001849_hilic_pos_FT6071', 'ST001428_hilic_pos_FT7882'), ('ST001849_hilic_pos_FT6967', 'ST001428_hilic_pos_FT8953'), ('ST001849_hilic_pos_FT6614', 'ST001428_hilic_pos_FT8577'), ('ST001849_hilic_pos_FT1988', 'ST001428_hilic_pos_FT1997'), ('ST001849_hilic_pos_FT1166', 'ST001428_hilic_pos_FT1040'), ('ST001849_hilic_pos_FT6793', 'ST001428_hilic_pos_FT8746'), ('ST001849_hilic_pos_FT5612', 'ST001428_hilic_pos_FT7210'), ('ST001849_hilic_pos_FT5067', 'ST001428_hilic_pos_FT6514'), ('ST001849_hilic_pos_FT5067', 'ST001428_hilic_pos_FT6525'), ('ST001849_hilic_pos_FT249', 'ST001428_hilic_pos_FT90'), ('ST001849_hilic_pos_FT5276', 'ST001428_hilic_pos_FT6708'), ('ST001849_hilic_pos_FT6924', 'ST001428_hilic_pos_FT8891'), ('ST001849_hilic_pos_FT6250', 'ST00

In [38]:
# Create a new graph to store the result
H = nx.Graph()

num_components = nx.number_connected_components(G)
# Find maximum cardinality matching for each connected component
for component in nx.connected_components(G):
    subgraph = G.subgraph(component)
    num_size = len(subgraph)
    matching = nx.bipartite.maximum_matching(subgraph)
    if num_size > 2:
        print(num_size)
        break
    H.add_edges_from(matching.items())

3


In [36]:
subgraph.edges()

EdgeView([('ST001428_hilic_pos_FT6525', 'ST001849_hilic_pos_FT5067'), ('ST001849_hilic_pos_FT5067', 'ST001428_hilic_pos_FT6514')])

In [39]:
matching

{'ST001428_hilic_pos_FT6525': 'ST001849_hilic_pos_FT5067',
 'ST001849_hilic_pos_FT5067': 'ST001428_hilic_pos_FT6525'}

In [30]:
num_components = nx.number_connected_components(G)
print("Number of connected components: {}".format(num_components))

Number of connected components: 931


In [24]:
matching

{'ST001849_hilic_pos_FT6855': 'ST001428_hilic_pos_FT8810',
 'ST001428_hilic_pos_FT8810': 'ST001849_hilic_pos_FT6855'}

In [23]:
H.edges()

EdgeView([('ST001428_hilic_pos_FT2109', 'ST001849_hilic_pos_FT2076'), ('ST001849_hilic_pos_FT2838', 'ST001428_hilic_pos_FT3141'), ('ST001428_hilic_pos_FT792', 'ST001849_hilic_pos_FT943'), ('ST001849_hilic_pos_FT3210', 'ST001428_hilic_pos_FT3687'), ('ST001849_hilic_pos_FT6071', 'ST001428_hilic_pos_FT7882'), ('ST001849_hilic_pos_FT6967', 'ST001428_hilic_pos_FT8953'), ('ST001849_hilic_pos_FT6614', 'ST001428_hilic_pos_FT8577'), ('ST001849_hilic_pos_FT1988', 'ST001428_hilic_pos_FT1997'), ('ST001428_hilic_pos_FT1040', 'ST001849_hilic_pos_FT1166'), ('ST001849_hilic_pos_FT6793', 'ST001428_hilic_pos_FT8746'), ('ST001849_hilic_pos_FT5612', 'ST001428_hilic_pos_FT7210'), ('ST001428_hilic_pos_FT6525', 'ST001849_hilic_pos_FT5067'), ('ST001428_hilic_pos_FT90', 'ST001849_hilic_pos_FT249'), ('ST001849_hilic_pos_FT5276', 'ST001428_hilic_pos_FT6708'), ('ST001428_hilic_pos_FT8891', 'ST001849_hilic_pos_FT6924'), ('ST001849_hilic_pos_FT6250', 'ST001428_hilic_pos_FT8106'), ('ST001849_hilic_pos_FT6698', 'ST00

In [15]:
len(nx.connected_components(G))


TypeError: object of type 'generator' has no len()

In [ ]:

matching = nx.bipartite.maximum_matching(G)

H = nx.Graph(matching)

In [13]:
len(nx.connected_components(G))

TypeError: object of type 'generator' has no len()

In [9]:
matching = nx.bipartite.maximum_matching(G)


AmbiguousSolution: Disconnected graph: Ambiguous solution for bipartite sets.